In [ ]:
import numpy as np
from cirq import (
    CNOT,
    I,
    X,
    Y,
    Z,
    LineQubit,
    Circuit,
    ops,
    unitary,
)
from mitiq import Executor, Observable, PauliString
from mitiq.interface.mitiq_cirq import compute_density_matrix
from mitiq.cdr._testing import random_x_z_cnot_circuit
from mitiq.pec.representations.learning import learn_biased_noise_parameters

Test with randomly generated circuit, compiled into (X, Z, CNOT) for `cdr.generate_training_circuits`

In [ ]:
circuit = random_x_z_cnot_circuit(
    LineQubit.range(3), n_moments=5, random_state=1
)
print(circuit)

Test with some example parameters (small offset to avoid long execution time)

In [ ]:
epsilon = 0.7
eta = 1000
gate = CNOT
offset = -0.1
observable = Observable(PauliString("XZ"), PauliString("YY"))

# Define biased noise channel
a = 1 - epsilon
b = epsilon * (3 * eta + 1) / (3 * (eta + 1))
c = epsilon / (3 * (eta + 1))

mix = [
    (a, unitary(I)),
    (b, unitary(Z)),
    (c, unitary(X)),
    (c, unitary(Y)),
]

def ideal_executor(circ: Circuit) -> np.ndarray:
    return compute_density_matrix(circ, noise_level=(0.0,))

def noisy_executor(circ: Circuit) -> np.ndarray:
    qreg = LineQubit.range(gate.num_qubits())
    return compute_density_matrix(
        circ, noise_model=ops.MixedUnitaryChannel(mix).on_each(*qreg)
    )

In [ ]:
[epsilon_opt, eta_opt] = learn_biased_noise_parameters(
    operation=gate,
    circuit=circuit,
    ideal_executor=Executor(ideal_executor),
    noisy_executor=Executor(noisy_executor),
    num_training_circuits=10,
    epsilon0=(1 + offset) * epsilon,
    eta0=(1 + offset) * eta,
    observable=observable
)

print(epsilon-epsilon_opt)
print(eta-eta_opt)